In [ ]:
from data_processing import process_mat_file
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from climate_extreme import ClimateExtreme
import climate_stats as cs
import scipy.stats as stats

from BucketModel import BucketModel, BucketModelOptimizer
from BucketModel.data_processing import preprocess_for_bucket_model, run_multiple_simulations
from BucketModel.bucket_model_plotter import *

from climate_simulation import run_model_for_future_climate, plot_climate_scenarios

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

import statsmodels.tsa.api as smt
import statsmodels.api as sm


from generate_future_climate import generate_future_climate

import os

from multiprocessing import Pool, cpu_count

import warnings

warnings.filterwarnings("ignore")

## (WEEK 1) Simulating ensembles with the WeaGETS weather generator.
### Reading in the input data and the output files from WeaGETS

In [ ]:
input_data_path = "/Users/cooper/Desktop/climate-impacts/data/Input_data.mat"
path_exp_first = "/Users/cooper/Desktop/climate-impacts/data/Exponential/Seperated_FirstOrder_Exp_1500(30).mat"
path_gamma_third = "/Users/cooper/Desktop/climate-impacts/data/Gamma/Seperated_ThirdOrder_Gamma_1500(30).mat"

present_data = process_mat_file(input_data_path)
exp_first_data = process_mat_file(path_exp_first)
gamma_third_data = process_mat_file(path_gamma_third)

In [ ]:
present_data

In [ ]:
exp_first_data

## Looking at climate statistics to compare generated data to observed data
### Comparing number of wet days (Precipitation > 0.1 mm) in the generated data to the observed data

In [ ]:
cs.plot_wet_days(
    present_data,
    exp_first_data,
    "/Users/cooper/Desktop/climate-impacts/images/wet_days.png",
)

### Comparing the Estimated Cumulative Distribution Function (ECDF) of the two datasets for Precipitation

In [ ]:
cs.plot_ECDF(
    observations=present_data,
    simulation=exp_first_data,
    column="Precipitation",
    xlabel="Precipitation (mm/day)",
)

### Comparing mean and standard deviation of Average Temperature values

In [ ]:
cs.plot_mean_and_std(observations=present_data, simulation=exp_first_data)

### Fit an extreme parameter distribution to the generated data and to the observed data and compare the confidence intervals of the parameters

In [ ]:
observed = ClimateExtreme(present_data)
generated = ClimateExtreme(exp_first_data)

In [ ]:
c, loc, scale, ci_lower, ci_upper = observed.fit_genextreme(
    "Precipitation", quantile=0.95, n_bootstrap=1000
)

print(f"GEV parameters: c={c:.4f}, loc={loc:.4f}, scale={scale:.4f}")
print("Confidence Intervals:")
print(f"c: ({ci_lower[0]:.4f}, {ci_upper[0]:.4f})")
print(f"loc: ({ci_lower[1]:.4f}, {ci_upper[1]:.4f})")
print(f"scale: ({ci_lower[2]:.4f}, {ci_upper[2]:.4f})")

In [ ]:
c, loc, scale, ci_lower, ci_upper = generated.fit_genextreme(
    "Precipitation", quantile=0.95, n_bootstrap=100
)

print(f"GEV parameters: c={c:.4f}, loc={loc:.4f}, scale={scale:.4f}")
print("Confidence Intervals:")
print(f"c: ({ci_lower[0]:.4f}, {ci_upper[0]:.4f})")
print(f"loc: ({ci_lower[1]:.4f}, {ci_upper[1]:.4f})")
print(f"scale: ({ci_lower[2]:.4f}, {ci_upper[2]:.4f})")

In [ ]:
observed.plot_fit_and_ci("Precipitation", "mm")
generated.plot_fit_and_ci("Precipitation", "mm")

#### Compare confidence intervals

In [ ]:
comparison = observed.compare_ci("Precipitation", generated, "Observed", "Generated")
print(comparison)

#### Check if confidence intervals overlap

In [ ]:
overlap = all(
    comparison["Observed_CI_upper"] >= comparison["Generated_CI_lower"]
) and all(comparison["Generated_CI_upper"] >= comparison["Observed_CI_lower"])

print(f"Confidence intervals {'overlap' if overlap else 'do not overlap'}")

#### Plot the distribution for visual comparison too

In [ ]:
observed.plot_extreme_comparison("Precipitation", generated)

### Perform a Truncated 2 sample KS test

In [ ]:
ks_stat, p_value = observed.truncated_ks_test("Precipitation", generated, quantile=0.95)
print(f"KS statistic: {ks_stat:.4f}, p-value: {p_value:.4f}")

### Running the Bucket Model
#### Preprocessing the data to match the format required by the Bucket Model


In [ ]:
processed_present_data = preprocess_for_bucket_model(present_data)
processed_exp_first_data = preprocess_for_bucket_model(exp_first_data)

In [ ]:
processed_present_data

In [ ]:
processed_exp_first_data

### Setting up the model for the catchment of Gsteig and the calibrated parameters from Assignment 2

In [ ]:
bucket_model = BucketModel(
    k=0.83, S_max=12.554, fr=0.111, rg=23.587, gauge_adj=0.267
)  # Parameters from Assignment 2

bucket_model.set_catchment_properties(
    lapse_rate=0.5 / 100,  # °C/m
    station_elevation=1638,  # m.a.s.l
    basin_elevation=2035,  # m.a.s.l
    snowmelt_temp_threshold=0,  # °C
    latitude=46.9,
)  # °N

#### Running the model for the present data and visualising the results

In [ ]:
results = bucket_model.run(data=processed_present_data)

In [ ]:
plot_water_balance(results=results, start_year="2010", end_year="2019")
plot_timeseries(
    results=results,
    start_year="2010",
    end_year="2019",
    monthly=True,
    plot_precipitation=True,
)
plot_timeseries(
    results=results,
    start_year="2010",
    end_year="2019",
    monthly=False,
    plot_precipitation=True,
)
plot_monthly_boxplot(results=results)

### Example of how to run the model for each Simulation individually

In [ ]:
multiple_results = run_multiple_simulations(
    preprocessed_simulated_data=processed_exp_first_data,
    bucket_model=bucket_model,
    n_simulations=50,
)

In [ ]:
monthly_mean, ci = group_by_month_with_ci(multiple_results)

plot_monthly_runoff_with_ci(monthly_mean, ci)

## (Week 2) Assessing the changes to streamflow in a future climate
### Generating future climate ensembles based on delta change method

In [ ]:
# generate_future_climate(
#     data=exp_first_data,
#     name="exp_first",
#     output_folder="/Users/cooper/Desktop/climate-impacts/data/FutureExponentialTest",
# )

generate_future_climate(
    data=gamma_third_data,
    name="gamma_third",
    output_folder="/Users/cooper/Desktop/climate-impacts/data/FutureGammaTest",
)

### Simulate future streamflow using the future climate ensemble and the Bucket

In [ ]:
future_data_folder = "/Users/cooper/Desktop/climate-impacts/data/FutureExponentialTest"
results = run_model_for_future_climate(
    future_data_folder=future_data_folder,
    bucket_model=bucket_model,
)

plot_climate_scenarios(results)

### Partition uncertainty

In [ ]:
from uncertainty_analysis import UncertaintyAnalysis

In [ ]:
path_to_future_data = "/Users/cooper/Desktop/climate-impacts/data/FutureExponentialTest/exp_first_CLMCOM-CCLM4-ECEARTH_RCP4.5.csv"


def prep_for_bucket_model(path: str) -> pd.DataFrame:
    """
    Preprocess future climate data for the bucket model

    Args:
        path (str): Path to the future climate data

    Returns:
        DataFrame: Future climate data prepared for the bucket model
    """

    date_cols = ["Year", "Month", "Day"]
    future_data = pd.read_csv(path, parse_dates={"Date": date_cols}, dayfirst=True)

    # Rename 'Precipitation' to 'P_mix'
    future_data.rename(columns={"Precipitation": "P_mix"}, inplace=True)

    # Set 'Date' as the index
    future_data.set_index("Date", inplace=True)

    return future_data

future_data = prep_for_bucket_model(path_to_future_data)

In [ ]:
def combine_climate_data(folder_path):
    """
    Combine climate data from multiple CSV files into a single DataFrame.

    This function reads climate data from CSV files in the specified folder, runs multiple simulations,
    and combines the results into a single DataFrame with annual mean streamflow, climate model, and scenario.

    Args:
        folder_path (str): Path to the folder containing the climate data CSV files.

    Returns:
        pd.DataFrame: Combined DataFrame with columns for 'Simulation', 'Year', 'Streamflow', 'Climate_Model', and 'Scenario'.
    """
    combined_data = []
    csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

    for filename in csv_files:
        try:
            file_path = os.path.join(folder_path, filename)
            future_data = prep_for_bucket_model(file_path)
            future_streamflow = run_multiple_simulations(
                future_data, bucket_model, n_simulations=50
            )
            future_streamflow["Streamflow"] = (
                future_streamflow["Q_s"] + future_streamflow["Q_gw"]
            )
            future_streamflow["Year"] = future_streamflow.index.year

            # Extract model and scenario from filename
            model, scenario = filename.rsplit("_", 2)[-2:]
            model = model.replace("-", "_")
            scenario = os.path.splitext(scenario)[0]

            annual_totals = (
                future_streamflow.groupby(["Simulation", "Year"])["Streamflow"]
                .sum()
                .reset_index()
            )

            annual_mean = (
                annual_totals.groupby(["Simulation"])["Streamflow"]
                .mean()
                .reset_index()
            )
            annual_mean["Climate_Model"] = model
            annual_mean["Scenario"] = scenario

            combined_data.append(annual_mean)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

    if combined_data:
        return pd.concat(combined_data, ignore_index=True)
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no data was processed

folder_path = "/Users/cooper/Desktop/climate-impacts/data/FutureGammaTest"
combined_df = combine_climate_data(folder_path)

In [ ]:
uncertainty = UncertaintyAnalysis(combined_df)

tu = uncertainty.calculate_tu('Streamflow')
eu, eu_partition = uncertainty.calculate_eu('Streamflow', tu)
cmu, cmu_partition = uncertainty.calculate_cmu('Streamflow', tu)
su, su_partition = uncertainty.calculate_su('Streamflow', tu)

print(f"Total Uncertainty: {tu}")
print(f"Emission Scenario Uncertainty: {eu} (Partition: {eu_partition})")
print(f"Climate Model Uncertainty: {cmu} (Partition: {cmu_partition})")
print(f"Stochastic Uncertainty: {su} (Partition: {su_partition})")